<a href="https://colab.research.google.com/github/alanwuha/ce7455-nlp/blob/master/assignment-1/word-level-language-model-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Two

Please make sure to run all cells in order.

## (i - ii) Setup

### Download wikitext-2 dataset

In [2]:
!rm -r data
!mkdir data
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip -P /content/data
!unzip /content/data/wikitext-2-v1.zip -d /content/data

--2020-02-09 03:01:57--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475746 (4.3M) [application/zip]
Saving to: ‘/content/data/wikitext-2-v1.zip’

wikitext-2-v1.zip   100%[===================>]   4.27M  18.5MB/s    in 0.2s    

2020-02-09 03:01:57 (18.5 MB/s) - ‘/content/data/wikitext-2-v1.zip’ saved [4475746/4475746]

Archive:  /content/data/wikitext-2-v1.zip
   creating: /content/data/wikitext-2/
  inflating: /content/data/wikitext-2/wiki.test.tokens  
  inflating: /content/data/wikitext-2/wiki.valid.tokens  
  inflating: /content/data/wikitext-2/wiki.train.tokens  


### Import libraries

In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
import argparse
import time
import os
from io import open

### Specify arg variables

In [0]:
data = './data/wikitext-2'  # location of the data corpus
args_model = 'FNN'          # type of recurrent net
emsize = 200                # size of word embeddings
nhid = 200                  # number of hidden units per layer
nlayers = 2                 # number of layers
lr = 20                     # initial learning rate
clip = 0.25                 # gradient clipping
epochs = 40                 # upper epoch limit
batch_size = 20             # batch size
bptt = 35                   # sequence length
dropout = 0.2               # dropout applied to layers (0 = no dropout)
tied = False                # tie the word embedding and softmax weights
seed = 1111                 # random seed
cuda = True                 # use CUDA
log_interval = 200          # report interval
save = 'model.pt'           # path to save the final model
onnx_export = ''            # path to export the final model in onnx format
nhead = 2                   # the number of heads in the encoder/decoder of the transformer model

### Data Preprocessing Classes

In [0]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'wiki.train.tokens'))
        self.valid = self.tokenize(os.path.join(path, 'wiki.valid.tokens'))
        self.test = self.tokenize(os.path.join(path, 'wiki.test.tokens'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)

        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

### Helper Methods

In [0]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)


# get_batch subdivides the source data into chunks of length bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    if args_model not in ['Transformer', 'FNN']:
        hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if args_model in ['Transformer', 'FNN']:
                output = model(data)
            else:
                output, hidden = model(data, hidden)
                hidden = repackage_hidden(hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    if args_model not in ['Transformer', 'FNN']:
        hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        # model.zero_grad()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output
        if args_model in ['Transformer', 'FNN']:
            output = model(data)
        else:
            hidden = repackage_hidden(hidden)
            output, hidden = model(data, hidden)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(output.view(-1, ntokens), targets)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # for p in model.parameters():
        #     p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.3f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, optimizer.param_groups[0]['lr'],
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def export_onnx(path, batch_size, seq_len):
    print('The model is also exported in ONNX format at {}'.
          format(os.path.realpath(args.onnx_export)))
    model.eval()
    dummy_input = torch.LongTensor(seq_len * batch_size).zero_().view(-1, batch_size).to(device)
    hidden = model.init_hidden(batch_size)
    torch.onnx.export(model, (dummy_input, hidden), path)

### Set torch seed and CUDA device

In [0]:
# Set the random seed manually for reproducibility.
torch.manual_seed(seed)

# Set cuda device
if torch.cuda.is_available():
    if not cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if cuda else "cpu")

### Load corpus

In [0]:
corpus = Corpus(data)

eval_batch_size = 10
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

ntokens = len(corpus.dictionary)

## (iii) Write a _class FNNModel(nn.Module)_.

The FNNModel should implement a language model with a feed-forward network architecture. It has a hidden layer with tanh architecture and the output layer is a Softmax layer. The output of the model for each input of (n-1) previous word indices are the probabilities of the |_V_| words in the vocabulary.

In [0]:
class FNNModel(nn.Module):
    """ Container module with an encoder, a feed forward module, and a decoder. """

    def __init__(self, ntokens, emsize, hidden_size, tie_weights=False):
        super(FNNModel, self).__init__()
        self.encoder = nn.Embedding(ntokens, emsize)
        self.hidden = nn.Linear(emsize, hidden_size)
        self.tanh = nn.Tanh()
        self.decoder = nn.Linear(hidden_size, ntokens)

        if tie_weights:
            if hidden_size != emsize:
                raise ValueError('When using the tied flag, hidden_size must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        # self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        emb = self.encoder(input)
        hid = self.hidden(emb)
        output = self.tanh(hid)
        decoded = self.decoder(output)
        return decoded

## (iv-1) Train the model with Adam optimizer (without sharing weights).

In [10]:
tied = False
save = 'model_untied.pt'
model = FNNModel(ntokens, emsize, nhid, tied).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())


# Loop over epochs.
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

| epoch   1 |   200/ 2983 batches | lr 0.001 | ms/batch 18.54 | loss  8.08 | ppl  3234.78
| epoch   1 |   400/ 2983 batches | lr 0.001 | ms/batch 17.40 | loss  6.59 | ppl   728.55
| epoch   1 |   600/ 2983 batches | lr 0.001 | ms/batch 17.45 | loss  6.38 | ppl   592.88
| epoch   1 |   800/ 2983 batches | lr 0.001 | ms/batch 17.50 | loss  6.31 | ppl   549.91
| epoch   1 |  1000/ 2983 batches | lr 0.001 | ms/batch 17.57 | loss  6.23 | ppl   507.14
| epoch   1 |  1200/ 2983 batches | lr 0.001 | ms/batch 17.58 | loss  6.22 | ppl   503.70
| epoch   1 |  1400/ 2983 batches | lr 0.001 | ms/batch 17.63 | loss  6.20 | ppl   490.41
| epoch   1 |  1600/ 2983 batches | lr 0.001 | ms/batch 17.67 | loss  6.18 | ppl   485.13
| epoch   1 |  1800/ 2983 batches | lr 0.001 | ms/batch 17.70 | loss  6.07 | ppl   430.87
| epoch   1 |  2000/ 2983 batches | lr 0.001 | ms/batch 17.75 | loss  6.10 | ppl   444.95
| epoch   1 |  2200/ 2983 batches | lr 0.001 | ms/batch 17.77 | loss  6.00 | ppl   401.94
| epoch   

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type FNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   2 |   200/ 2983 batches | lr 0.001 | ms/batch 18.07 | loss  5.73 | ppl   307.82
| epoch   2 |   400/ 2983 batches | lr 0.001 | ms/batch 18.04 | loss  5.68 | ppl   292.37
| epoch   2 |   600/ 2983 batches | lr 0.001 | ms/batch 18.06 | loss  5.56 | ppl   258.84
| epoch   2 |   800/ 2983 batches | lr 0.001 | ms/batch 18.12 | loss  5.59 | ppl   268.52
| epoch   2 |  1000/ 2983 batches | lr 0.001 | ms/batch 18.15 | loss  5.56 | ppl   259.41
| epoch   2 |  1200/ 2983 batches | lr 0.001 | ms/batch 18.20 | loss  5.58 | ppl   264.33
| epoch   2 |  1400/ 2983 batches | lr 0.001 | ms/batch 18.25 | loss  5.61 | ppl   272.79
| epoch   2 |  1600/ 2983 batches | lr 0.001 | ms/batch 18.27 | loss  5.61 | ppl   271.92
| epoch   2 |  1800/ 2983 batches | lr 0.001 | ms/batch 18.29 | loss  5.52 | ppl   250.46
| epoch   2 |  2000/ 2983 batches | lr 0.001 | ms/batch 18.36 | loss  5.58 | ppl   264.73
| epoch   2 |  2200/ 2983 batches | lr 0.001 | ms/batch 18.40 | loss  5.48 | ppl   239.13
| epoch   

## (v-1) Show the perplexity score on the test set. You should select your best model based on the _valid_ set.

In [11]:
# Load the best saved model.
save = 'model_untied.pt'
with open(save, 'rb') as f:
    model = torch.load(f)

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  5.73 | test ppl   308.45


## (iv-2) Train the model with Adam optimizer, but now with sharing the input (look-up matrix) and output layer embeddings (final layer weights).

In [12]:
tied = True
save = 'model_tied.pt'
model = FNNModel(ntokens, emsize, nhid, tied).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())


# Loop over epochs.
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

| epoch   1 |   200/ 2983 batches | lr 0.001 | ms/batch 17.04 | loss 14.22 | ppl 1502783.63
| epoch   1 |   400/ 2983 batches | lr 0.001 | ms/batch 16.91 | loss  8.03 | ppl  3073.87
| epoch   1 |   600/ 2983 batches | lr 0.001 | ms/batch 16.89 | loss  7.31 | ppl  1488.76
| epoch   1 |   800/ 2983 batches | lr 0.001 | ms/batch 16.90 | loss  7.13 | ppl  1243.80
| epoch   1 |  1000/ 2983 batches | lr 0.001 | ms/batch 16.89 | loss  7.00 | ppl  1101.65
| epoch   1 |  1200/ 2983 batches | lr 0.001 | ms/batch 16.88 | loss  6.97 | ppl  1067.11
| epoch   1 |  1400/ 2983 batches | lr 0.001 | ms/batch 16.90 | loss  6.91 | ppl  1003.37
| epoch   1 |  1600/ 2983 batches | lr 0.001 | ms/batch 16.91 | loss  6.89 | ppl   986.13
| epoch   1 |  1800/ 2983 batches | lr 0.001 | ms/batch 16.88 | loss  6.79 | ppl   885.45
| epoch   1 |  2000/ 2983 batches | lr 0.001 | ms/batch 16.91 | loss  6.80 | ppl   899.18
| epoch   1 |  2200/ 2983 batches | lr 0.001 | ms/batch 16.89 | loss  6.71 | ppl   820.16
| epoch 

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type FNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   2 |   200/ 2983 batches | lr 0.001 | ms/batch 16.99 | loss  6.54 | ppl   692.33
| epoch   2 |   400/ 2983 batches | lr 0.001 | ms/batch 16.90 | loss  6.47 | ppl   644.98
| epoch   2 |   600/ 2983 batches | lr 0.001 | ms/batch 16.88 | loss  6.36 | ppl   579.72
| epoch   2 |   800/ 2983 batches | lr 0.001 | ms/batch 16.89 | loss  6.40 | ppl   599.72
| epoch   2 |  1000/ 2983 batches | lr 0.001 | ms/batch 16.88 | loss  6.36 | ppl   576.65
| epoch   2 |  1200/ 2983 batches | lr 0.001 | ms/batch 16.90 | loss  6.38 | ppl   588.80
| epoch   2 |  1400/ 2983 batches | lr 0.001 | ms/batch 16.89 | loss  6.38 | ppl   589.49
| epoch   2 |  1600/ 2983 batches | lr 0.001 | ms/batch 16.88 | loss  6.38 | ppl   590.09
| epoch   2 |  1800/ 2983 batches | lr 0.001 | ms/batch 16.88 | loss  6.28 | ppl   532.73
| epoch   2 |  2000/ 2983 batches | lr 0.001 | ms/batch 16.91 | loss  6.32 | ppl   553.87
| epoch   2 |  2200/ 2983 batches | lr 0.001 | ms/batch 16.89 | loss  6.23 | ppl   507.36
| epoch   

## (v-2) Show the perplexity score on the test set. You should select your best model based on the _valid_ set.

In [13]:
# Load the best saved model.
save = 'model_tied.pt'
with open(save, 'rb') as f:
    model = torch.load(f)

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  5.92 | test ppl   372.39


## (vii) Adapt generate.py so that you can generate texts using your language model (FNNModel)

In [14]:
data = './data/wikitext-2'          # location of the data corpus
checkpoint = './model_untied.pt'    # model checkpoint to use
outf = 'generated.txt'              # output file for generated text
words = 1000                        # number of words to generate
seed = 1111                         # random seed
cuda = True                         # use CUDA
temperature = 1.0                   # temperature - higher will increase diversity
log_interval = 100                  # reporting interval

with open(checkpoint, 'rb') as f:
  model = torch.load(f).to(device)
model.eval()

input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with open(outf, 'w') as outf:
  with torch.no_grad(): # no tracking history
    for i in range(words):
      output = model(input)
      word_weights = output.squeeze().div(temperature).exp().cpu()
      word_idx = torch.multinomial(word_weights, 1)[0]
      input.fill_(word_idx)

      word = corpus.dictionary.idx2word[word_idx]

      outf.write(word + ('\n' if i % 20 == 19 else ' '))

      if i % log_interval == 0:
        print('| Generated {}/{} words'.format(i, words))

| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words


In [15]:
!cat generated.txt

, salute a shipment to Innocent was attracted the <unk> , the world at the 5th Division I forced the
conclave for the assistance are pink and pending a fascinated by Echmarcach <unk> , and financial crisis ( at <unk>
( 12 nmi of puzzles , which was a 50 million . Hergé is regulated . This contains political forecastle
Gannit to have been being <unk> , <unk> of Heraclea Pontica , while turned into motor physician as from disbanding
their first reported the creation of the 766th Regiment and deploying @-@ 46 is still Berliner " . <eos> unprecedented
development team , gable including Eurasian in baseball Weird Edwards , Walpole known as manager Frank Flower and " for
sale of number of <unk> Agreement supports fold @-@ lying for expression , 2012 , a significant men ensued .
<eos> = = <eos> <eos> = <eos> Stevens ' , he describes the Second World in Osmond , the "
<unk> " As a 16 , one of the next year , who yet to David vase Jha , other
reptiles the second people who dealt , nightclub : 1

## (viii) In your opinion, which computation/operation is the most expensive one in inference or forward pass? Can you think of ways to improve this? Is yes, please mention.

## (ix) Report the Spearman correlation for the input embeddings.

Notice that the model also learns word vectors (input and output layer embeddings) as a byproduct. One way to evaluate the trained word vectors is to __measure the cosine similarity__ between pairs of words, and then __report the correlation with the similarity scores given by humans__. For this exercise, use the dataset available [here](http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/) and report the Spearman correlation for the input embeddings. Exclude any pair if it is not in the embedding matrix.

### Download wordsimilarity-353 dataset

In [16]:
!wget http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip -P /content/data
!unzip /content/data/wordsim353.zip -d /content/data/wordsim353

--2020-02-09 04:16:58--  http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip
Resolving www.cs.technion.ac.il (www.cs.technion.ac.il)... 132.68.32.15
Connecting to www.cs.technion.ac.il (www.cs.technion.ac.il)|132.68.32.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23257 (23K) [application/zip]
Saving to: ‘/content/data/wordsim353.zip’

wordsim353.zip      100%[===================>]  22.71K  65.2KB/s    in 0.3s    

2020-02-09 04:16:59 (65.2 KB/s) - ‘/content/data/wordsim353.zip’ saved [23257/23257]

Archive:  /content/data/wordsim353.zip
  inflating: /content/data/wordsim353/combined.csv  
  inflating: /content/data/wordsim353/set1.csv  
  inflating: /content/data/wordsim353/set2.csv  
  inflating: /content/data/wordsim353/combined.tab  
  inflating: /content/data/wordsim353/set1.tab  
  inflating: /content/data/wordsim353/set2.tab  
  inflating: /content/data/wordsim353/instructions.txt  


### Without sharing weights

#### Load model

In [30]:
device = torch.device('cpu')
checkpoint = 'model_untied.pt'
with open(checkpoint, 'rb') as f:
  model = torch.load(f)
  model = model.to(device)
model

FNNModel(
  (encoder): Embedding(33278, 200)
  (hidden): Linear(in_features=200, out_features=200, bias=True)
  (tanh): Tanh()
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
)

#### Prepare Spearman rank-order correlation data table

In [31]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

# Read csv
df = pd.read_csv('/content/data/wordsim353/combined.csv')

# Get word indices for Word 1
word_1 = df['Word 1']
df['Indices 1'] = word_1.apply(lambda x: corpus.dictionary.word2idx[x] if x in corpus.dictionary.word2idx else None)

# Get word indices for Word 2
word_2 = df['Word 2']
df['Indices 2'] = word_2.apply(lambda x: corpus.dictionary.word2idx[x] if x in corpus.dictionary.word2idx else None)

# Drop rows with nan values and reset index
df = df.dropna().reset_index(drop=True)

# Set indices to integer data type
df['Indices 1'] = df['Indices 1'].astype('int')
df['Indices 2'] = df['Indices 2'].astype('int')

# Get word embeddings from trained model
embedding_1 = df['Indices 1'].apply(lambda x: model.encoder.weight[x].detach().numpy())
embedding_2 = df['Indices 2'].apply(lambda x: model.encoder.weight[x].detach().numpy())

# Compute cosine similarity from word embeddings
cosine_similarity = []
for t1, t2 in zip(*(embedding_1, embedding_2)):
  cos_sim = t1.dot(t2) / (norm(t1) * norm(t2))
  cosine_similarity.append(cos_sim)
df['Cosine Similarity'] = pd.Series(cosine_similarity)

# Sort by Human (mean) and assign rank values
df = df.sort_values(['Human (mean)']).reset_index(drop=True)
df['Rank 1'] = np.arange(len(df)) + 1

# Sort by Cosine Similarity
df = df.sort_values(['Cosine Similarity']).reset_index(drop=True)
df['Rank 2'] = np.arange(len(df)) + 1

# Compute d and d^2 where d = difference between ranks and d^2 = difference squared
df['d'] = df['Rank 1'] - df['Rank 2']
df['d^2'] = df['d'] ** 2

# Reindex dataframe columns
df = df.reindex(columns=['Word 1', 'Word 2', 'Human (mean)', 'Cosine Similarity', 'Rank 1', 'Rank 2', 'd', 'd^2'])

df

,Word 1,Word 2,Human (mean),Cosine Similarity,Rank 1,Rank 2,d,d^2
0,street,place,6.44,-0.160048,173,1,172,29584
1,opera,industry,2.63,-0.142855,33,2,31,961
2,grocery,money,5.94,-0.138252,140,3,137,18769
3,music,project,3.63,-0.135753,61,4,57,3249
4,bird,cock,7.10,-0.130913,214,5,209,43681
...,...,...,...,...,...,...,...,...
311,marathon,sprint,7.47,0.173335,236,312,-76,5776
312,glass,metal,5.56,0.185894,122,313,-191,36481
313,family,planning,6.25,0.190070,160,314,-154,23716
314,television,radio,6.77,0.233762,193,315,-122,14884


#### Calculate Spearman correlation

The Spearman correlation is a nonparametric measure of the monotonicity of the relationship between two datasets. Unlike the Pearson correlation, the Spearman correlation does not assume that both datasets are normally distributed. Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply an exact monotonic relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreses.

The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Spearman correlation at least as extreme as the one computed from these datasets. The p-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so.

In [32]:
n = len(df)
corr = 1 - (6 * df['d^2'].sum()) / (n * (n**2 - 1))

print('The spearman correlation value is', corr)

The spearman correlation value is -0.06990566701463963


#### Cross check answer with scipy.stats.spearmanr

In [33]:
import scipy.stats as s

spearman_corr, spearman_p  = s.spearmanr(df['Rank 1'], df['Rank 2'])

print('The spearman correlation value is', spearman_corr)

The spearman correlation value is -0.06990566701463959


And we get the same result!

#### The value being close to zero shows that the correlation between `Human (mean)` and `Cosine Similiarity` is __low__.

### With sharing weights

#### Loading model (with sharing weights)

In [34]:
device = torch.device('cpu')
checkpoint = 'model_tied.pt'
with open(checkpoint, 'rb') as f:
  model = torch.load(f)
  model = model.to(device)
model

FNNModel(
  (encoder): Embedding(33278, 200)
  (hidden): Linear(in_features=200, out_features=200, bias=True)
  (tanh): Tanh()
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
)

#### Prepare Spearman rank-order correlation data table

In [35]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

# Read csv
df = pd.read_csv('/content/data/wordsim353/combined.csv')

# Get word indices for Word 1
word_1 = df['Word 1']
df['Indices 1'] = word_1.apply(lambda x: corpus.dictionary.word2idx[x] if x in corpus.dictionary.word2idx else None)

# Get word indices for Word 2
word_2 = df['Word 2']
df['Indices 2'] = word_2.apply(lambda x: corpus.dictionary.word2idx[x] if x in corpus.dictionary.word2idx else None)

# Drop rows with nan values and reset index
df = df.dropna().reset_index(drop=True)

# Set indices to integer data type
df['Indices 1'] = df['Indices 1'].astype('int')
df['Indices 2'] = df['Indices 2'].astype('int')

# Get word embeddings from trained model
embedding_1 = df['Indices 1'].apply(lambda x: model.encoder.weight[x].detach().numpy())
embedding_2 = df['Indices 2'].apply(lambda x: model.encoder.weight[x].detach().numpy())

# Compute cosine similarity from word embeddings
cosine_similarity = []
for t1, t2 in zip(*(embedding_1, embedding_2)):
  cos_sim = t1.dot(t2) / (norm(t1) * norm(t2))
  cosine_similarity.append(cos_sim)
df['Cosine Similarity'] = pd.Series(cosine_similarity)

# Sort by Human (mean) and assign rank values
df = df.sort_values(['Human (mean)']).reset_index(drop=True)
df['Rank 1'] = np.arange(len(df)) + 1

# Sort by Cosine Similarity
df = df.sort_values(['Cosine Similarity']).reset_index(drop=True)
df['Rank 2'] = np.arange(len(df)) + 1

# Compute d and d^2 where d = difference between ranks and d^2 = difference squared
df['d'] = df['Rank 1'] - df['Rank 2']
df['d^2'] = df['d'] ** 2

# Reindex dataframe columns
df = df.reindex(columns=['Word 1', 'Word 2', 'Human (mean)', 'Cosine Similarity', 'Rank 1', 'Rank 2', 'd', 'd^2'])

df

,Word 1,Word 2,Human (mean),Cosine Similarity,Rank 1,Rank 2,d,d^2
0,discovery,space,6.34,-0.175937,166,1,165,27225
1,noon,string,0.54,-0.152691,3,2,1,1
2,size,prominence,5.31,-0.141904,116,3,113,12769
3,king,cabbage,0.23,-0.127920,1,4,-3,9
4,planet,star,8.45,-0.124253,296,5,291,84681
...,...,...,...,...,...,...,...,...
311,tiger,feline,8.00,0.198444,271,312,-41,1681
312,midday,noon,9.29,0.206150,313,313,0,0
313,man,woman,8.30,0.206269,287,314,-27,729
314,deployment,withdrawal,5.88,0.220321,134,315,-181,32761


#### Calculate Spearman correlation

The Spearman correlation is a nonparametric measure of the monotonicity of the relationship between two datasets. Unlike the Pearson correlation, the Spearman correlation does not assume that both datasets are normally distributed. Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply an exact monotonic relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreses.

The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Spearman correlation at least as extreme as the one computed from these datasets. The p-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so.

In [36]:
n = len(df)
corr = 1 - (6 * df['d^2'].sum()) / (n * (n**2 - 1))

print('The spearman correlation value is', corr)

The spearman correlation value is 0.0448573215973288


#### Cross check answer with scipy.stats.spearmanr

In [37]:
import scipy.stats as s

spearman_corr, spearman_p  = s.spearmanr(df['Rank 1'], df['Rank 2'])

print('The spearman correlation value is', spearman_corr)

The spearman correlation value is 0.04485732159732879


And we get the same result!

#### The value being close to zero shows that the correlation between `Human (mean)` and `Cosine Similiarity` is __low__.